The notebook is for converting SLURP into Cantonese and upload to HF.

In [1]:
import transformers
import datasets
import pandas as pd
from preprocessing import hybrid_split

def getColsForTrain(example):

    intent = example["intent"]
    speech = example["sentence"]
    tkn = ["0"] * len(hybrid_split(speech.lower()))
    ner_tag = "'" + "".join(tkn)

    result = {
        "Speech": speech,
        "Intent": intent,
        "NER_Tag": ner_tag,
    }

    return result


def import_jsonl(jsonl_path, train_data_file="train.jsonl", test_data_file="test.jsonl"):
    """Import a jsonl file and return a list of dictionaries."""
    ds_train = datasets.load_dataset(jsonl_path, data_files=train_data_file, split="train")
    filtered_train = datasets.Dataset.from_dict({
        "intent": ds_train["intent"],
        "sentence": ds_train["sentence"]
    })
    ds_test = datasets.load_dataset(jsonl_path, data_files=test_data_file, split="train")
    filter_test = datasets.Dataset.from_dict({
        "intent": ds_test["intent"],
        "sentence": ds_test["sentence"]
    })
    mapped_train = filtered_train.map(getColsForTrain, remove_columns=["sentence", "intent"])
    mapped_test = filter_test.map(getColsForTrain, remove_columns=["sentence", "intent"])
    ds = datasets.DatasetDict(
        {
            "train": mapped_train,
            "test": mapped_test,
        }
    )
    return ds

slurp_en = import_jsonl("training_data/slurp/")

print(slurp_en["train"][36])


c:\Users\borisPMC\Documents\GitHub\Medic_Grabber\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2974/2974 [00:00<00:00, 17919.66 examples/s]

{'Speech': 'play audiobook', 'Intent': 'play_audiobook', 'NER_Tag': "'00"}


Label Count (Train Set)

In [ ]:
list = [i["intent"] for i in slurp_en["train"]]
label_list = set(list)
label_count = [sum(1 for i in slurp_en["train"] if i["intent"] == label) for label in label_list]
print(label_list)
print(label_count)

# {'quirky', 'alarm_query', 'iot_cleaning', 'music', 'lists_query', 'email_querycontact', 'post', 'query', 'alarm_remove', 'qa_currency', 'hue_lightup', 'iot_wemo_off', 'lists_remove', 'querycontact', 'general_greet', 'datetime_convert', 'sendemail', 'calendar_query', 'currency', 'email_sendemail', 'greet', 'social_query', 'podcasts', 'audio_volume_up', 'recommendation_movies', 'volume_other', 'cooking_query', 'calendar_set', 'convert', 'calendar_remove', 'remove', 'transport_query', 'cooking_recipe', 'definition', 'audio_volume_mute', 'audio_volume_down', 'factoid', 'qa_factoid', 'traffic', 'hue_lightdim', 'joke', 'play_music', 'events', 'createoradd', 'play_audiobook', 'qa_maths', 'play_podcasts', 'music_likeness', 'news_query', 'cleaning', 'audio_volume_other', 'lists_createoradd', 'alarm_set', 'qa_stock', 'general_joke', 'play_radio', 'transport_taxi', 'iot_hue_lightoff', 'wemo_off', 'transport_ticket', 'takeaway_order', 'recommendation_events', 'music_settings', 'takeaway_query', 'play_game', 'hue_lightoff', 'qa_definition', 'settings', 'recommendation_locations', 'transport_traffic', 'email_query', 'email_addcontact', 'iot_hue_lighton', 'iot_coffee', 'general_quirky', 'addcontact', 'set', 'coffee', 'iot_hue_lightchange', 'iot_hue_lightdim', 'game', 'radio', 'iot_hue_lightup', 'music_dislikeness', 'datetime_query', 'social_post', 'wemo_on', 'weather_query', 'music_query', 'iot_wemo_on', 'ticket'}
# [9, 130, 92, 19, 197, 126, 4, 45, 78, 137, 2, 50, 162, 1, 23, 51, 7, 558, 5, 347, 2, 106, 4, 110, 70, 1, 4, 804, 1, 307, 7, 227, 207, 1, 110, 52, 9, 535, 2, 2, 2, 620, 1, 5, 150, 78, 189, 113, 500, 1, 17, 172, 182, 152, 70, 277, 100, 147, 2, 126, 135, 189, 50, 120, 108, 6, 266, 1, 173, 115, 415, 53, 22, 120, 546, 1, 6, 4, 125, 74, 4, 6, 74, 14, 342, 279, 1, 559, 150, 47, 1]

Translate Data Set

In [ ]:
import asyncio
from googletrans import Translator

test_set = slurp_en["train"][:10]["Speech"]

async def translate_text():
    async with Translator() as translator:
        translations = await translator.translate(test_set, dest='Cantonese')
        for translation in translations:
            print(translation.origin, ' -> ', translation.text)

asyncio.run(translate_text())

RuntimeError: asyncio.run() cannot be called from a running event loop

: 